<h1 align='center'>Synthetic Data Generation and Unsloth Tutorial</h1>

## 📚 Table of Contents:

- [Synthetic Data Kit: Data Generation](#synthetic-data-generation)
- [Unsloth: Fine-Tuning and saving the model](#fine-tuning)

## Synthetic Data Generation

In this section, we use the CLI from synthetic-data-kit to generate datasets

### Testing Synthetic Data Kit Command

Please make sure you are running vllm by opening a terminal and typing `vllm serve Unsloth/Llama-3.3-70B-Instruct   --port 8001   --max-model-len 48000   --gpu-memory-utilization 0.85`

### Exploring Synthetic Data Kit CLI

This command displays the help menu for the `synthetic-data-kit` CLI tool, showing available commands:
- **system-check**: Verify LLM provider server is running
- **ingest**: Parse documents (PDF, HTML, YouTube, etc.) into clean text
- **create**: Generate synthetic content (Q&A pairs, instructions, etc.) using LLM
- **curate**: Filter and clean generated content based on quality scores
- **save-as**: Convert data to different formats (fine-tuning format, JSON, etc.)
- **server**: Launch web interface for the toolkit

In [1]:
!synthetic-data-kit --help

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
                                                                                
 Usage: synthetic-data-kit [OPTIONS] COMMAND [ARGS]...                          
                                                                                
 A toolkit for preparing synthetic datasets for fine-tuning LLMs                
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --config              -c      PATH  Path to configuration file               │
│ --install-completion                Install completion for the current       │
│                                     shell.                                  

### Verifying LLM Server Status

This command checks if the vLLM server is running and accessible at `http://localhost:8001/v1`. It displays:
- Server status and endpoint
- Available models (here: Unsloth/Llama-3.3-70B-Instruct)
- Model configuration (max context length: 48000 tokens)

The system is configured to use the vLLM provider as specified in `tutorial_config.yaml`.

In [2]:
!synthetic-data-kit -c tutorial_config.yaml system-check

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: tutorial_config.yaml
Config has LLM provider set to: vllm
Environment variable check:
API_ENDPOINT_KEY: Not found
get_llm_provider returning: vllm
L vLLM server is not available at http://localhost:8001/v1
Error: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded 
with url: /v1/models (Caused by 
NewConnectionError('<urllib3.connection.HTTPConnection object at 
0x7f6c06938440>: Failed to establish a new connection: [Errno 111] Connection 
refused'))
m⠋ Checking vLLM server at http://localhost:8001/v1...
To start the server, run:
vllm serve Unsloth/Llama-3.3-70B-Instruct --port 8001
⠋ Checking vLLM server at http://localhost:8001/v1...v1...


### Creating Project Directory Structure

This command creates a well-organized directory structure for the logical reasoning project:
- `sources/`: Store original source documents (PDFs, etc.)
- `data/input/`: Input files for processing
- `data/parsed/`: Parsed text files after document ingestion
- `data/generated/`: Generated synthetic Q&A pairs
- `data/curated/`: Quality-filtered data after curation
- `data/final/`: Final formatted data ready for fine-tuning

In [3]:
!bash -c 'mkdir -p logical_reasoning/{sources,data/{input,parsed,generated,curated,final}}'

### Navigating to Project Directory

Changes the current working directory to `logical_reasoning/` where all subsequent operations will take place.

In [4]:
cd logical_reasoning

/workspace/AAIPL/logical_reasoning


### Downloading Source Documents

Downloads two PDF documents related to logical reasoning and liar/truth puzzles:
1. "Logical Reasoning" textbook from CSU Sacramento
2. "Liar and Truth Teller Puzzles" from UMass

These documents will serve as the knowledge base for generating synthetic training data. The `-q` flag runs wget in quiet mode, and `--show-progress` displays a progress bar.

In [5]:
!wget -P sources/ -q --show-progress   "https://www.csus.edu/faculty/d/dowden/_internal/_documents/logical-reasoning-12.pdf"   "https://people.cs.umass.edu/~pthomas/solutions/Liar_Truth.pdf"

logical-reasoning-1 100%[===================>]   5.52M  1.49MB/s    in 4.0s    
Liar_Truth.pdf      100%[===================>] 327.61K  --.-KB/s    in 0.1s    


### Copying Source Files to Input Directory

Copies all downloaded source documents from `sources/` to `data/input/` to prepare them for the ingestion pipeline.

In [6]:
!cp sources/* data/input/

### Ingesting and Parsing Documents

This command processes the PDF files in `data/input/` using the synthetic-data-kit's **ingest** command:
- Extracts text content from PDFs
- Cleans and normalizes the text
- Saves parsed text files to `data/parsed/`

The output shows successful processing of 2 PDF files (Liar_Truth.pdf and logical-reasoning-12.pdf).

In [7]:
!synthetic-data-kit ingest ./data/input/

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Processing directory: ./data/input/
Found 2 supported files to process
✓ Liar_Truth.pdf
✓ logical-reasoning-12.pdf

Processing Summary:
Total files: 2
Successful: 2
Failed: 0
✅ All files processed successfully!


### Generating Synthetic Q&A Pairs

This command uses the synthetic-data-kit's **create** command to generate Q&A pairs from the parsed text:
- Reads parsed text files from `data/parsed/`
- Uses the vLLM provider with Llama-3.3-70B-Instruct model
- Generates 50 Q&A pairs per file (`--num-pairs 50`)
- Type is set to `qa` for question-answer pair generation
- Outputs are saved to `data/generated/`

The process chunks the text and generates questions with corresponding answers. This took about 10 minutes for the full run. Use `--verbose` flag to see detailed progress or reduce `--num-pairs` for faster testing.

Note: This will take about 10 minutes, set `--verbose` flag to see progress or reduce the `num-pairs` for a faster test

In [8]:
!synthetic-data-kit -c ../tutorial_config.yaml create ./data/parsed/ --type qa --num-pairs 50

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: ../tutorial_config.yaml
Config has LLM provider set to: vllm
get_llm_provider returning: vllm
🔗 Using vllm provider
❌ Error: VLLM server not available at http://localhost:8001/v1
Please start the VLLM server with:
vllm serve Unsloth/Llama-3.3-70B-Instruct


### Curating and Quality Filtering

This command uses the **curate** function to filter generated Q&A pairs based on quality:
- Evaluates each Q&A pair using quality metrics
- Filters pairs with quality score above threshold (7.0/10)
- Removes low-quality, inconsistent, or malformed pairs
- Saves curated data to `data/curated/`

This ensures only high-quality synthetic data is used for fine-tuning.

Note: This will also take about 10 minutes.

In [9]:
!synthetic-data-kit -c ../tutorial_config.yaml curate ./data/generated/ --threshold 7.0

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: ../tutorial_config.yaml
Config has LLM provider set to: vllm
get_llm_provider returning: vllm
🔗 Using vllm provider
❌ Error: VLLM server not available at http://localhost:8001/v1
Please start the VLLM server with:
vllm serve Unsloth/Llama-3.3-70B-Instruct


### Converting to Fine-Tuning Format

This command uses the **save-as** function to convert curated Q&A pairs to fine-tuning format:
- Reads curated JSON files from `data/curated/`
- Converts to format `ft` (fine-tuning format with messages structure)
- Outputs are saved to `data/final/` with proper conversation format
- The resulting format is compatible with standard fine-tuning pipelines

Successfully converted 2 files to fine-tuning format.

In [10]:
!synthetic-data-kit save-as ./data/curated/ --format ft

Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.12/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Processing directory: ./data/curated/ for format conversion to ft
No supported files found in ./data/curated/
For save-as: looking for .json files with cleaned QA pairs
✅ All files converted successfully!


### Loading and Converting Data to HuggingFace Dataset

This cell performs comprehensive data processing:

1. **Finding Files**: Locates all JSON files in `data/final/` directory
2. **Loading Data**: Reads each JSON file containing fine-tuning formatted data
3. **Format Conversion**: Extracts user and assistant messages from the fine-tuning format
4. **Structuring Conversations**: Creates a standardized conversation format with role-content pairs
5. **Creating Dataset**: Converts the processed data into a HuggingFace Dataset object

The output shows 74 total conversations were successfully loaded and formatted. The preview displays a sample conversation showing either a knight-and-knave logic puzzle with its solution or a deduction puzzle about the Logical Reasoning book.

In [16]:
import json
import glob
from pathlib import Path
from datasets import Dataset

# ===== CONFIGURATION =====
data_dir = "./data/final"  # Change this to your data directory

# ===== STEP 1: Find all FT files =====
data_path = Path(data_dir)
ft_files = glob.glob(str(data_path / "*.json"))

# ===== STEP 2: Load and convert all files =====
all_data = []

for file_path in ft_files:
    # Load the JSON file
    with open(file_path, 'r') as f:
        ft_data = json.load(f)
    
    # Convert each item
    for item in ft_data:
        if 'messages' not in item:
            continue
        
        # Extract only user and assistant messages
        conversation = []
        for msg in item['messages']:
            if msg['role'] == 'user' or msg['role'] == 'assistant':
                conversation.append({
                    "role": msg['role'],
                    "content": msg['content']
                })
        
        # Add to our data if we have at least one exchange
        if len(conversation) > 0:
            all_data.append({
                "conversations": conversation
            })

print(f"\n🎯 Total conversations: {len(all_data)}")

# ===== STEP 3: Create HuggingFace Dataset =====
dataset = Dataset.from_list(all_data)

# ===== STEP 4: Preview the data =====
print(json.dumps(dataset[0], indent=2))


🎯 Total conversations: 0


IndexError: Invalid key: 0 is out of bounds for size 0

## Fine-Tuning

### Note: Please remember to shutdown the vLLM instance!

### Importing Standard Libraries

Imports essential Python libraries for fine-tuning:
- `os`, `json`, `glob`: File system operations and JSON handling
- `torch`: PyTorch deep learning framework
- `shutil`: File operations
- `Path`: Path manipulation
- `Dataset`: HuggingFace datasets library for data handling

In [13]:
import os
import json
import glob
import torch
import shutil
from pathlib import Path
from datasets import Dataset

### Importing Unsloth and Training Libraries

Imports specialized libraries for efficient fine-tuning:
- `FastLanguageModel` from Unsloth: Optimized model loading and training
- `get_chat_template`, `standardize_sharegpt`, `train_on_responses_only`: Chat formatting utilities
- `SFTConfig`, `SFTTrainer`: Supervised fine-tuning configuration and trainer from TRL
- `DataCollatorForSeq2Seq`: Handles batching and padding for sequence-to-sequence training

In [14]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 02-14 10:31:49 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Setup Unsloth model and tokenizer for ROCm without bitsandbytes

### Loading Llama-3.3-70B Model with LoRA

This cell sets up the model for efficient fine-tuning on AMD ROCm hardware:

**Model Configuration:**
- Model: Llama-3.3-70B-Instruct (70 billion parameters)
- Data type: bfloat16 for ROCm compatibility
- No quantization (load_in_4bit=False) to avoid bitsandbytes dependency
- Max sequence length: 1024 tokens

**LoRA (Low-Rank Adaptation) Configuration:**
- Rank (r): 64 - Higher rank for the large 70B model
- Target modules: All attention and MLP layers (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj)
- LoRA alpha: 64
- Dropout: 0 (no dropout)
- Gradient checkpointing: "unsloth" for memory efficiency

LoRA enables efficient fine-tuning by only training small adapter layers instead of the entire 70B model, making it feasible to train on a single AMD MI300X GPU with 192GB HBM3 memory.

In [15]:
max_seq_length = 1024
dtype = torch.bfloat16  # Explicit bfloat16 for ROCm
load_in_4bit = False  

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.3-70B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # Explicit for ROCm
    trust_remote_code=True,
)

print(f"✅ Loaded: Llama-3.3-70B-Instruct (bfloat16, ROCm compatible)")

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # Higher rank for 70B model
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=64,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 30] Read-only file system: '/root/.cache/huggingface/hub'
[2026-02-14 10:31:52] ERROR file_download.py:1556: Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 30] Read-only file system: '/root/.cache/huggingface/hub'


Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.


RuntimeError: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.

### Preparing Dataset with Chat Template

This cell formats the dataset for fine-tuning:

**Steps:**
1. **Set Chat Template**: Applies Llama-3.1 chat template formatting
2. **Configure Padding**: Sets pad token to eos token if not already set
3. **Format Conversations**: The `formatting_prompts_func` function:
   - Takes raw conversations from the dataset
   - Applies the chat template to format them properly
   - Validates conversation structure (list of dicts with role/content)
   - Filters out malformed conversations
4. **Standardize Format**: Uses `standardize_sharegpt` to normalize the data structure
5. **Apply Formatting**: Maps the formatting function across all examples
6. **Remove Empty**: Filters out any empty or invalid formatted texts

The output shows 74 valid examples were successfully prepared. A sample of the formatted text is displayed, showing the proper Llama-3.1 chat template structure with system, user, and assistant headers.

In [ ]:
"""Prepare dataset with proper chat template and tensor compatibility"""
print("🔧 Preparing dataset for training...")

# Set chat template
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Formatting function that ensures proper tensor conversion
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = []
    
    for convo in convos:
        # Ensure conversation is in correct format
        if isinstance(convo, list) and all(isinstance(msg, dict) for msg in convo):
            text = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
            texts.append(text)
        else:
            print(f"⚠️  Skipping malformed conversation: {type(convo)}")
            continue
    
    return {"text": texts}

dataset = standardize_sharegpt(dataset)

dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)

dataset = dataset.filter(lambda x: len(x["text"].strip()) > 0)

print(f"✅ Prepared {len(dataset)} valid examples for training")

# Show sample
if len(dataset) > 0:
    print(f"📝 Sample formatted text:")
    print(dataset["text"][0][:200] + "...")

### Training the Model with ROCm-Optimized Settings

This cell configures and executes the fine-tuning process:

**Training Configuration (SFTConfig):**
- **Batch size**: 64 per device - leveraging the AMD MI300X's massive 192GB HBM3 memory
- **Gradient accumulation**: 1 step
- **Warmup**: 5 steps
- **Epochs**: 1 full pass through the dataset
- **Learning rate**: 1e-4
- **Optimizer**: adamw_8bit for memory efficiency
- **Precision**: bf16 (bfloat16) for ROCm
- **Gradient checkpointing**: Enabled for memory efficiency

**Special Training Mode:**
Uses `train_on_responses_only` to compute loss only on the assistant's responses, not on the user's questions. This focuses the model on learning to generate accurate answers rather than memorizing the input format.

**Key Features:**
- DataCollatorForSeq2Seq handles variable-length sequences with proper padding
- No packing to preserve conversation structure
- Single dataloader worker for ROCm stability
- Gradient checkpointing via Unsloth for memory optimization

The model is then trained on the 74 logical reasoning conversations.

In [ ]:
"""Train model with ROCm-optimized settings"""
# Ensure tokenizer has proper padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Setup trainer with ROCm-friendly settings and proper data handling
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=32,  # 🚀 MI300X can handle this with 192GB HBM3!
        gradient_accumulation_steps=2,   # Effective batch size = 32*2 = 64
        warmup_steps=5,
        num_train_epochs=1,
        learning_rate=1e-4,
        logging_steps=1,
        optim="adamw_8bit",  # Pure torch optimizer
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="logical_reasoning_rocm_outputs",
        report_to="none",
        bf16=True,
        dataloader_pin_memory=False,
        remove_unused_columns=True,  # Remove unused columns to avoid tensor issues
        gradient_checkpointing=True,
        dataloader_num_workers=0,  # Single worker for ROCm stability
    ),
)

# Train only on responses
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

FastLanguageModel.for_training(model)
trainer_stats = trainer.train()


trainer_stats = trainer.train()

### Saving the Fine-Tuned Model

This cell saves the trained model in two formats:

1. **LoRA Adapters** (`logical_reasoning_rocm_lora/`):
   - Saves only the trained LoRA adapter weights (lightweight, ~few hundred MB)
   - Can be loaded later with the base model
   - Useful for sharing or deploying with the original base model

2. **Merged Model** (`logical_reasoning_rocm_merged/`):
   - Merges LoRA adapters back into the base model
   - Creates a standalone model with all weights
   - Saved in 16-bit precision for better quality
   - Ready for immediate inference without loading adapters

Both formats include the tokenizer configuration. The merged model is production-ready and can be used directly for generating answers to logical reasoning questions.

In [ ]:
"""Save the trained model"""
print("\n💾 SAVING ROCM-TRAINED MODEL")

# Save LoRA adapters
lora_path = "logical_reasoning_rocm_lora"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"✅ LoRA adapters saved to: {lora_path}")

# Save merged model
merged_path = "logical_reasoning_rocm_merged"
print("🔄 Saving merged model...")
model.save_pretrained_merged(merged_path, tokenizer, save_method="merged_16bit")
print(f"✅ Merged model saved to: {merged_path}")

print(f"\n🎉 ROCM MODEL READY!")

### Testing the Fine-Tuned Model
You can do a quick test to check that the model is working.

In [ ]:
"""Test the fine-tuned model with inference"""
# Switch model to inference mode
FastLanguageModel.for_inference(model)

# Test question - a classic knight/knave logic puzzle
test_question = "A says 'B is a knave.' B says 'A and I are different types.' What are A and B?"

# Format the prompt using the chat template
messages = [{"role": "user", "content": test_question}]
input_text = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True  # Adds assistant header so model knows to respond
)

# Tokenize and move to GPU
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.1,      # Low temperature for more deterministic logical reasoning
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
)

# Decode only the generated part (exclude the input prompt)
response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

print(f"Question: {test_question}\n")
print(f"Answer: {response}")

In [ ]:
#fin